In [1]:
import tensorflow as tf
import sys
sys.path.append('/nas/longleaf/home/siyangj/NiftyNet/')
import niftynet as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk
import os
import re
import nibabel as nib
import sklearn as skl
import pandas as pd

data_path = '/proj/NIRAL/users/siyangj/myData/min_cut_outside/'
model_path = '/proj/NIRAL/users/siyangj/model_10112050/'

inference_path = model_path+'inference_17000_10112050/'


INFO:tensorflow:TensorFlow version 1.9.0
CRITICAL:tensorflow:Optional Python module cv2 not found, please install cv2 and retry if the application fails.
INFO:tensorflow:Available Image Loaders:
['nibabel', 'skimage', 'pillow', 'simpleitk', 'dummy'].


In [2]:
inf_re = re.compile(r'inference.*')
im_re = re.compile(r'.*\.nii\.gz')

In [3]:
inf_dirs = [f for f in os.listdir(model_path) 
            if (os.path.isdir(os.path.join(model_path, f)) and inf_re.match(f) is not None)]

In [4]:
eval_df = pd.DataFrame(columns=['model','inf_dir','test_im','test_11','test_12','test_21','test_22',
                                '1','2','3','4','5','6','7','8'],dtype=object)

In [5]:
for i in range (1,9):
    eval_df[str(i)] = eval_df[str(i)].astype(object)

In [36]:
for d in inf_dirs:
    inf_dir = os.path.join(model_path,d)
    inf_ims = {f[0]:os.path.join(inf_dir,f) for f in os.listdir(inf_dir) 
            if (os.path.isfile(os.path.join(inf_dir, f)) and im_re.match(f) is not None)}
    l = len(eval_df.index)
    eval_df.loc[l,'inf_dir']=inf_dir
    for im_num,im in inf_ims.items():
        true_im = os.path.join(data_path,'seg/',im_num+'.nrrd')
        true_arr = sitk.GetArrayFromImage(sitk.ReadImage(true_im))
        #print(true_arr.shape)
        inf_arr = np.swapaxes(nib.load(im).get_fdata().squeeze(),0,2)
        #print(inf_arr.shape)
        c_mat = skl.metrics.confusion_matrix(true_arr.flatten(),inf_arr.flatten())
        p_mat = c_mat.T / c_mat.astype(np.float).sum(axis=1)
        eval_df.loc[l,im_num]=[p_mat.T]

In [44]:
p_mat.T

array([[9.99981809e-01, 0.00000000e+00, 0.00000000e+00, 1.81908217e-05],
       [1.10726644e-04, 9.26571626e-01, 5.85024221e-02, 1.48152249e-02],
       [3.09740345e-03, 3.08778949e-01, 6.49880988e-01, 3.82426597e-02],
       [5.00030348e-02, 2.88910919e-02, 9.71230299e-02, 8.23982843e-01]])

In [35]:
c_mat.T / c_mat.astype(np.float).sum(axis=0)

array([[9.88487697e-01, 0.00000000e+00, 0.00000000e+00, 9.57406068e-05],
       [3.99594012e-05, 6.69796294e-01, 5.62643097e-02, 2.84668738e-02],
       [1.59637808e-03, 3.18773762e-01, 8.92614877e-01, 1.04942343e-01],
       [9.87596602e-03, 1.14299436e-02, 5.11208136e-02, 8.66495043e-01]])

In [11]:
c_mat.astype(np.float).sum(axis=1)

array([494755., 180625., 257958.,  98854.])

In [38]:
eval_df

,inf_dir,1,2,3,4,5,6,7,8
0,/proj/NIRAL/users/siyangj/model_10112050/infer...,"[[[494753, 0, 0, 2], [12, 165979, 12174, 2460]...","[[[536955, 13, 1258, 2569], [20, 149200, 28375...","[[[589984, 0, 0, 2], [0, 155789, 10834, 4516],...","[[[578525, 0, 0, 2], [0, 185316, 11230, 2803],...","[[[525103, 0, 1, 38], [0, 135436, 6210, 206], ...","[[[631945, 0, 0, 27], [0, 127211, 3957, 99], [...","[[[637542, 0, 0, 21], [0, 136002, 5708, 229], ...","[[[651380, 0, 5, 106], [0, 131362, 4133, 243],..."
1,/proj/NIRAL/users/siyangj/model_10112050/infer...,"[[[494749, 0, 0, 6], [12, 161691, 17547, 1375]...","[[[536927, 0, 3633, 235], [22, 155156, 24801, ...","[[[589968, 0, 6, 12], [34, 154315, 14995, 1795...","[[[578510, 0, 7, 10], [6, 179281, 16864, 3198]...","[[[525063, 0, 14, 65], [0, 133305, 8170, 377],...","[[[631920, 0, 6, 46], [0, 125971, 5280, 16], [...","[[[637471, 0, 39, 53], [1, 132978, 8941, 19], ...","[[[651419, 0, 31, 41], [0, 128564, 7133, 41], ..."
2,/proj/NIRAL/users/siyangj/model_10112050/infer...,"[[[494746, 0, 0, 9], [20, 167362, 10567, 2676]...","[[[538478, 0, 1730, 587], [13, 167891, 10840, ...","[[[589972, 0, 6, 8], [40, 149448, 18661, 2990]...","[[[578520, 0, 2, 5], [5, 165440, 29308, 4596],...","[[[525103, 0, 9, 30], [0, 132298, 9219, 335], ...","[[[631930, 0, 2, 40], [0, 126306, 4934, 27], [...","[[[637497, 0, 5, 61], [4, 133521, 8366, 48], [...","[[[651443, 0, 9, 39], [1, 127465, 8223, 49], [..."
3,/proj/NIRAL/users/siyangj/model_10112050/infer...,NaN,NaN,NaN,NaN,"[[[525103, 0, 1, 38], [0, 135436, 6210, 206], ...",NaN,NaN,"[[[651380, 0, 5, 106], [0, 131362, 4133, 243],..."
4,/proj/NIRAL/users/siyangj/model_10112050/infer...,"[[[0.9999959575951733, 0.0, 0.0, 2.02318570821...","[[[0.9928993426344548, 7.181883973902138e-05, ...","[[[0.9999966100890529, 0.0, 0.0, 3.13180188221...","[[[0.9999965429444089, 0.0, 0.0, 3.00246201885...","[[[0.9999257343727982, 0.0, 3.4715110446123883...","[[[0.9999572765882033, 0.0, 0.0, 0.00061206447...","[[[0.9999670620785711, 0.0, 0.0, 0.00049900199...","[[[0.9998296215910888, 0.0, 2.5290844714213454..."
5,/proj/NIRAL/users/siyangj/model_10112050/infer...,"[[[0.9999878727855201, 0.0, 0.0, 6.06955712464...","[[[0.9928475670078311, 0.0, 0.0153315074505302...","[[[0.9999694908014767, 0.0, 2.895669044332693e...","[[[0.999970615027475, 0.0, 3.729275881174615e-...","[[[0.9998495644987451, 0.0, 4.860115462457344e...","[[[0.9999177178735766, 0.0, 2.668564312399929e...","[[[0.9998557005346923, 0.0, 0.0001851799094042...","[[[0.9998894842753008, 0.0, 0.0001568032372281..."
6,/proj/NIRAL/users/siyangj/model_10112050/infer...,"[[[0.9999818091782802, 0.0, 0.0, 9.10433568697...","[[[0.9957155668968833, 0.0, 0.0073007178335858...","[[[0.9999762706233707, 0.0, 2.895669044332693e...","[[[0.9999879003054308, 0.0, 1.0655073946213187...","[[[0.9999257343727982, 0.0, 3.12435994015115e-...","[[[0.9999335413594274, 0.0, 8.89521437466643e-...","[[[0.9998964808183662, 0.0, 2.3741014026191086...","[[[0.9999263228502006, 0.0, 4.5523520485584216..."
7,/proj/NIRAL/users/siyangj/model_10112050/infer...,"[[[0.9999959575951733, 0.0, 0.0, 4.04240482663...","[[[0.9928993426344548, 2.4038683789606042e-05,...","[[[0.9999966100890529, 0.0, 0.0, 3.38991094703...","[[[0.9999965429444089, 0.0, 0.0, 3.45705559118...","[[[0.9999257343727982, 0.0, 1.9042468513278313...","[[[0.9999572765882033, 0.0, 0.0, 4.27234117967...","[[[0.9999670620785711, 0.0, 0.0, 3.29379214289...","[[[0.9998296215910888, 0.0, 7.674703104110418e..."
8,/proj/NIRAL/users/siyangj/model_10112050/infer...,"[[[0.9999878727855201, 0.0, 0.0, 1.21272144798...","[[[0.9928475670078311, 0.0, 0.0067178875544337...","[[[0.9999694908014767, 0.0, 1.0169732841118263...","[[[0.999970615027475, 0.0, 1.209969456913852e-...","[[[0.9998495644987451, 0.0, 2.6659455918589638...","[[[0.9999177178735766, 0.0, 9.494091510383372e...","[[[0.9998557005346923, 0.0, 6.11704255108907e-...","[[[0.9998894842753008, 0.0, 4.758315924548459e..."
9,/proj/NIRAL/users/siyangj/model_10112050/infer...

In [42]:
str(round(1.0000))

'1'

In [46]:
os.path.join('123/','456')

'123/456'